# Shipwrecks of Massachusetts

## Package Imports

In [13]:
import pandas as pd
import geopandas as gpd
import requests
import fiona
import shapely

# Enable KML fiona driver support (disabled by default)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

## Retrieve Data

The two datasets used for this project are:

1. The API response from [DiveSites.com](http://api.divesites.com/docs/) for a set of recreational SCUBA diving sites within 50 nautical miles of Boston, MA
2. A KML file from [Wikipedia](https://en.wikipedia.org/wiki/List_of_shipwrecks_of_Massachusetts) with coordinates for a subset of their listed ship wrecks of Massachusetts

In [14]:
# Get data from DiveSites.com API
ds_url = "http://api.divesites.com/?mode=sites&lat=42.35843&lng=-71.05977&dist=50"
r_ds = requests.get(ds_url)
ds_json = r_ds.json()

In [15]:
ds_json

{'request': {'str': None,
  'timestamp': 1606584803,
  'loc': {'lat': '42.35843', 'lng': '-71.05977'},
  'mode': 'sites',
  'dist': '50',
  'api': 1},
 'sites': [{'currents': None,
   'distance': '8.85',
   'hazards': None,
   'lat': '42.4196',
   'name': 'Chimneys',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '21163',
   'equipment': None,
   'lng': '-70.9074'},
  {'currents': None,
   'distance': '8.87',
   'hazards': None,
   'lat': '42.4227',
   'name': '40 Steps',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '21164',
   'equipment': None,
   'lng': '-70.9093'},
  {'currents': None,
   'distance': '8.98',
   'hazards': None,
   'lat': '42.3738',
   'name': 'Romance',
   'water': None,
   'marinelife': None,
   'description': None,
   'maxdepth': None,
   'mindepth': None,
   'predive': None,
   'id': '

Dive sites are stored in a list of objects with form:

```py
{'currents': None,
 'distance': '8.85',
 'hazards': None,
 'lat': '42.4196',
 'name': 'Chimneys',
 'water': None,
 'marinelife': None,
 'description': None,
 'maxdepth': None,
 'mindepth': None,
 'predive': None,
 'id': '21163',
 'equipment': None,
 'lng': '-70.9074'}
```

In [16]:
len(ds_json['sites'])

34

In [8]:
# Get MA ship wreck file from Wikipedia
# sw_url = "https://tools.wmflabs.org/kmlexport?article=List_of_shipwrecks_of_Massachusetts"

# r_sw = requests.get(sw_url)

In [18]:
df = gpd.read_file('./Data/MA_Shipwrecks_Wikipedia.kml', driver='KML')

# Drop Z dimension of polygons that occurs often in kml
df.geometry = df.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))

df.head()

,Name,Description,geometry
0,Dixie Sword,"<br>Source: Wikipedia article <a href=""https:/...",POINT Z (-69.98056 41.54944 0.00000)
1,SS James Longstreet,"<br>Source: Wikipedia article <a href=""https:/...",POINT Z (-70.04424 41.83146 0.00000)
2,USS Merrimack (1798),"<br>Source: Wikipedia article <a href=""https:/...",POINT Z (-69.74900 42.12600 0.00000)
3,Pendleton,"<br>Source: Wikipedia article <a href=""https:/...",POINT Z (-69.96250 41.58611 0.00000)
4,HMS Somerset,"<br>Source: Wikipedia article <a href=""https:/...",POINT Z (-70.14806 42.07611 0.00000)


In [19]:
len(df)

6

In [20]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

,Name,Description,geometry
0,Dixie Sword,"<br>Source: Wikipedia article <a href=""https:/...",POINT (-69.98056 41.54944)
1,SS James Longstreet,"<br>Source: Wikipedia article <a href=""https:/...",POINT (-70.04424 41.83146)
2,USS Merrimack (1798),"<br>Source: Wikipedia article <a href=""https:/...",POINT (-69.74900 42.12600)
3,Pendleton,"<br>Source: Wikipedia article <a href=""https:/...",POINT (-69.96250 41.58611)
4,HMS Somerset,"<br>Source: Wikipedia article <a href=""https:/...",POINT (-70.14806 42.07611)


In [ ]:


r = requests.get("http://watchthemed.net/index.php/json/layer/9")

# convert the page content to bytes
f =  fiona.BytesCollection(bytes(r.content))

# empty GeoDataFrame
df = gp.GeoDataFrame()

# iterate over layers
for layer in fiona.listlayers(f.path):
    s = gp.read_file(f.path, driver='KML', layer=layer)
    df = df.append(s, ignore_index=True)


df.head()